In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# librerias propias
import metodosINE.especificas as me

In [52]:
def month(mes, es_numerico = True):
    meses = {
        1: "january", 2: "february", 3: "march", 4: "april", 
        5: "may", 6: "june", 7: "july", 8: "august", 
        9: "setpember", 10: "october", 11: "november", 12: "december"
    }

    if es_numerico:
        if mes in meses:
            return meses[mes]
    else: 
        mes = mes.lower()
        meses_invertido = {v: k for k, v in meses.items()}
        if mes in meses_invertido:
            return meses_invertido[mes]

In [53]:
chromedriver = r"C:\Users\daalvarado\.cache\selenium\chromedriver\win64\127.0.6533.88\chromedriver.exe"
url = 'https://www.statsf1.com/en/statistiques/pilote.aspx'
url_base = 'https://www.statsf1.com'

In [54]:
service = Service(chromedriver)
driver = webdriver.Chrome(service=service)
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
links = soup.find_all('a', href=True)
englishlinks = [link['href'] for link in links if '/en/' in link['href']]

In [55]:
useful_links = []
pattern = '/statistiques/pilote/'
for link in englishlinks:
    match1 = re.search(pattern, link)
    if match1:
        cadena = link.replace('/en' + pattern, '')
        cadena = cadena.replace('/', '_')
        cadena = cadena.replace('.aspx', '')
        #print(f'Link: {cadena} -> {link}')
        useful_links.append((url_base + link, cadena))

In [56]:
soups = []
for link, name in useful_links:
    # solicitamos la información a la página
    driver.get(link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    rows = soup.find_all('tr')
    data = []
    for row in rows[1:]:
        cell = row.find_all('td')
        lcell = len(cell)
        line = []
        for i in range(lcell):
            line.append(cell[i].get_text(separator=' ', strip=True))
        data.append(tuple(line))
    soups.append((name, data))

In [60]:
with pd.ExcelWriter('statsf1.xlsx', engine='xlsxwriter') as writer:
    for i in range(len(soups)):
        try:
            #df = pd.DataFrame(data[1:], columns=data[0])
            name = soups[i][0]
            df = pd.DataFrame(soups[i][1][1:], columns=soups[i][1][0])
            df.to_excel(writer, sheet_name=name, index=False)
            print(f'{name} agregado exitosamente...')
        except Exception as e:
            print(f'Error en {name}...')
            continue

champion_nombre agregado exitosamente...
champion_chronologie agregado exitosamente...
champion_age agregado exitosamente...
Error en champion_doyen...
Error en champion_consecutif...
champion_gp-avant agregado exitosamente...
champion_saison-avant agregado exitosamente...
champion_victoire agregado exitosamente...
champion_ecart agregado exitosamente...
champion_constructeur-different agregado exitosamente...
champion_moteur-different agregado exitosamente...
champion_vice-champion agregado exitosamente...
champion_podium agregado exitosamente...
champion_meilleur-classement agregado exitosamente...
Error en victoire_nombre...
victoire_chronologie agregado exitosamente...
victoire_chronologie-record agregado exitosamente...
Error en victoire_age...
victoire_jour-anniversaire agregado exitosamente...
Error en victoire_age-nation...
Error en victoire_doyen...
Error en victoire_consecutive...
victoire_consecutive-debut agregado exitosamente...
victoire_derniere agregado exitosamente...
E

In [156]:
driver.get('https://www.statsf1.com/en/statistiques/pilote/sprint/entete-tour.aspx')

In [55]:
html_champion = driver.page_source
soup_champion = BeautifulSoup(html_champion, 'html.parser')

In [57]:
data = []
rows = soup_champion.find_all('tr')
for row in rows[1:]:
    cells = row.find_all('td')
    if len(cells) == 4:
        number = cells[0].get_text(strip=True)
        date = cells[1].get_text(strip=True)
        year = cells[2].get_text(strip=True)
        driver = cells[3].get_text(strip=True)
        data.append((number, date, year, driver))
dataframe = pd.DataFrame(data[1:], columns=data[0])
dataframe[['WeekDay', 'Day', 'Month']] = dataframe['Date'].str.split('[, ]+', expand=True)
dataframe = dataframe.drop('Date', axis=1)
dataframe = dataframe.dropna()

In [157]:
html_sprint = driver.page_source
soup_sprint = BeautifulSoup(html_sprint, 'html.parser')

In [158]:
soup_sprint

<html id="ctl00_html" lang="en"><head>
<!-- Google Analytics -->
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=G-VV2RVNBDQ6"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  gtag('config', 'G-VV2RVNBDQ6');
</script>
<title>
	Statistics Drivers - Sprint - Lap led • STATS F1
</title><meta charset="utf-8"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="Sport" name="category"/><meta content="STATS F1" name="author"/><meta content="#176AC7" name="theme-color"/><link href="https://www.statsf1.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/><link href="https://www.statsf1.com/apple-touch-icon.png" rel="apple-touch-icon"/><link href="https://code.jquery.com/ui/1.13.3/themes/cupertino/jquery-ui.css" rel="stylesheet" typ